# Question answering with multiple PDFs using DeciLM-7B-instruct, LangChain and FAISS

In [1]:
%%capture
!pip install langchain transformers accelerate tiktoken openai gradio torch accelerate \
safetensors sentence-transformers faiss-gpu bitsandbytes pypdf typing-extensions

In [2]:
%%capture
!pip uninstall typing-extensions --yes
!pip install typing-extensions

Downloading sample files

In [3]:
!gdown 1GZoh1XDNbaH97HUgWwr-lCDmHq4a8yTX

Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1GZoh1XDNbaH97HUgWwr-lCDmHq4a8yTX

but Gdown can't. Please check connections and permissions.


In [4]:
!gdown 1IAMpfIir7iK-4EAI221gMMXlINzLxsNK

Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1IAMpfIir7iK-4EAI221gMMXlINzLxsNK

but Gdown can't. Please check connections and permissions.


# The Processes involved in creating the chatbot

## Import required packages

In [5]:
%%capture
!pip install PyPDF2

In [6]:

import torch
import PyPDF2 # pdf reader
import time
from pypdf import PdfReader
from io import BytesIO
from langchain.prompts import PromptTemplate # for custom prompt specification
from langchain.text_splitter import RecursiveCharacterTextSplitter # splitter for chunks
from langchain.embeddings import HuggingFaceEmbeddings # embeddings
from langchain.vectorstores import FAISS # vector store database
from langchain.chains import RetrievalQA # qa and retriever chain
from langchain.memory import ConversationBufferMemory # for model's memoy on past conversations
from langchain.document_loaders import PyPDFDirectoryLoader # loader fo files from firectory

from langchain.llms.huggingface_pipeline import HuggingFacePipeline # pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig



In [7]:
# Initialize some variables

In [8]:
CHUNK_SIZE = 1000
# Using HuggingFaceEmbeddings with the chosen embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",model_kwargs = {"device": "cuda"})

# transformer model configuration
# this massively model's precision for memory effieciency
# The models accuacy is reduced.
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

## The LLM in use - `DeciLM-7B-instruct`

In [ ]:
tensor_1 = torch.rand (4,4)

In [ ]:
model_id = "Deci/DeciLM-7B-instruct" # model repo id
device = 'cuda' # Run on gpu if available else run on cpu

#
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             trust_remote_code=True,
                                             device_map = "auto",
                                             quantization_config=quant_config)

# create a pipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                return_full_text = True,
                max_new_tokens=200,
                repetition_penalty = 1.1,
                num_beams=5,
                no_repeat_ngram_size=4,
                early_stopping=True)

llm = HuggingFacePipeline(pipeline=pipe)

## Loading the PDFs from directory with PyPDFDirectoryLoader

We load the files with PyPDFDirectoryLoader DocumentLoader which returns Document object. Document is an object with page_content and metdata. The metadata can be sources, page numbers etc.


In [9]:
pdf_paths = "/content/"

loader = PyPDFDirectoryLoader(
    path= pdf_paths,
    glob="*.pdf"
)
documents=loader.load()

print(len(documents))
documents[0] # display four documents

0


IndexError: list index out of range

In [ ]:
documents[0].page_content, documents[0].metadata

## Splitting the documents

We can not pass the entire documents into the language model since they have the maximum number of tokens. for instance the DeciLM-7B-instruct has 4096. Large chunks also make it hard for the model to search for relevant information from very prmpts.

So we need to split the documents into small text chunks

We split the documents with `RecursiveCharacterTextSplitter`. We sent the

* `chunk_size = 1000`(1000 characters) and

* `chunk_overlap =  200`(200 characters of overlap between chunks. The overlap helps mitigate the possibility of separating a statement from important context related to it)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE,
                                                chunk_overlap=100)

splits = text_splitter.split_documents(documents)

# length of all splits

print(f"We have, {len(splits)} chunks in memory")

## Creating a Vector store

Now that we've got 119 text chunks in memory, we need to store and index them so that we
can search them later in our Question answering app.

We use a Vector store database for this. In this case FAISS. We embedd the text chunks and sttore them. This
helps for easier similarity check between the stored chunks and our query to the bot.


We use Hugging Face embeddings.

In [10]:
vectorstore_db = FAISS.from_documents(splits, embeddings) # create vector db for similarity search

NameError: name 'splits' is not defined

## Next we create a Question Answering chain.

First, we want an application that let's the user ask a question, searches for documents relevant to that question, passes the retrieved documents and initial question to a model, and finally returns an answer.

To allow this, We use a `Retriever` interface which wraps an index that can return relevant documents given a string query. It uses the similarity search capabilities of a vector store to facillitate retrieval. In that case, We will convert the `vectorstore_db` we created above into a retriever.

In [ ]:

# performs a similarity check and returns the top K embeddings
# that are similar to the question’s embeddings
retriever = vectorstore_db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

For instance based on our PDFs, we can get the relevant documents based on a query.

PDF 1 : "Attention is all You need"

In [11]:
retrieved_relevant_docs = retriever.get_relevant_documents(
    "Based on the Deep unlearning pdf, what does class unlearning mean?"
)

print(f"Retrieved documents: {len(retrieved_relevant_docs)}")
f"Page content of first document:\n {retrieved_relevant_docs[0].page_content}"


NameError: name 'retriever' is not defined

PDF 2 : "Tex-to Music Generation"

In [ ]:
retrieved_relevant_docs = retriever.get_relevant_documents(
    """Based on Eight things to know about large language models pdf,
    where do much of the expense of developing an LLM go?"""
)

print(f"Retrieved documents: {len(retrieved_relevant_docs)}")
f"Page content of first document:\n {retrieved_relevant_docs[0].page_content}"

PDF 3 : "APIs"

Since we have seen how the retriever is working, We will implement the  Question answeing chain using `RetrievalQA`
from Langchain. However improve user experience and the quality of the conversation with the chatbot,
we need to implement `Memory`

### Implementing memory

Large Language Models are by default `stateless` meaning each incoming query is processed independently of other interactions. The only thing that exists for a stateless agent is the current input, nothing else.

For our chatbot, remembering previous interactions is important. So we will allow the DeciLM-7B-instruct LLM Lto remember previous interactions with the user.


We will use the `ConversationBufferMemory` from LangChain.

First, we create a custom prompt

### Create a custom prompt

In [12]:
custom_prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context and answer the question at the end.
If you don't know the answer just say you d not know an do not try to make up the answer nor try to use outside souces to answer. Keep the answer as concise as possible.
Context= {context}
History = {history}
Question= {question}
Helpful Answer:
"""

prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=["question", "context", "history"])

## Final question answeing chain with memory

In [ ]:
qa_chain_with_memory = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff',
                                                   retriever = vectorstore_db.as_retriever(),
                                                   return_source_documents = True,
                                                   chain_type_kwargs = {"verbose": True,
                                                                        "prompt": prompt,
                                                                        "memory": ConversationBufferMemory(
                                                                            input_key="question",
                                                                            memory_key="history",
                                                                            return_messages=True)})



Try some queries now and see the answers

In [13]:
query = "Based on the Deep unlearning pdf, what is deep unlearning?"
qa_chain_with_memory({"query": query})

NameError: name 'qa_chain_with_memory' is not defined

Ask a follow-up question to see if the model has history of previous queries

In [ ]:
qa_chain_with_memory({"query": "What are the key challenges involved?"})

### To check and verify the memory/chat history

In [14]:
print(qa_chain_with_memory.combine_documents_chain.memory)

NameError: name 'qa_chain_with_memory' is not defined

# Creating the chatbot application with Gradio

## Building chatbot GUI with gradio

In [ ]:
import gradio as gr


In [ ]:

def load_llm():
    # Loads the  DeciLM-7B-instruct llm when called
    model_id = "Deci/DeciLM-7B-instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id,
                                                 trust_remote_code=True,
                                                 device_map = "auto",
                                                 quantization_config=quant_config)
    pipe = pipeline("text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    temperature=0,
                    num_beams=5,
                    no_repeat_ngram_size=4,
                    early_stopping=True,
                    max_new_tokens=100,
                )

    llm = HuggingFacePipeline(pipeline=pipe)
    return llm

def add_text(history, text):
  # Adding user query to the chatbot and chain
  # use history with curent user question
  if not text:
      raise gr.Error('Enter text')
  history = history + [(text, '')]
  return history

def upload_file(files):
  # Loads files when the file upload button is clicked
  # Displays them on the File window
  # print(type(file))
  return files

def process_file(files):

    """Function reads each loaded file, and extracts text from each of their pages
    The extracted text is store in the 'text variable which is the passed to the splitter
    to make smaller chunks necessary for easier information retrieval and adhere to max-tokens(4096) of DeciLM-7B-instruct"""

    pdf_text = ""
    for file in files:
      pdf = PyPDF2.PdfReader(file.name)
      for page in pdf.pages:
          pdf_text += page.extract_text()


    # split into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=200)
    splits = text_splitter.create_documents([pdf_text])

    # create a FAISS vector store db
    # embedd the chunks and store in the db
    vectorstore_db = FAISS.from_documents(splits, embeddings)

    #create a custom prompt
    custom_prompt_template = """You have been given the following documents to answer the user's question.
    If you do not have information from the files given to answer the questions just say I don't have information from the given files to answer. Do not try to make up an answer.
    Context: {context}
    History: {history}
    Question: {question}

    Helpful answer:
    """
    prompt = PromptTemplate(template=custom_prompt_template, input_variables=["question", "context", "history"])

    # set QA chain with memory
    qa_chain_with_memory = RetrievalQA.from_chain_type(llm=load_llm(),
                                                       chain_type='stuff',
                                                       return_source_documents=True,
                                                       retriever=vectorstore_db.as_retriever(),
                                                       chain_type_kwargs={"verbose": True,
                                                                          "prompt": prompt,
                                                                          "memory": ConversationBufferMemory(
                                                                              input_key="question",
                                                                              memory_key="history",
                                                                              return_messages=True) })
    # get answers
    return qa_chain_with_memory

def generate_bot_response(history,query, btn):
  """Fiunction takes the query, history and inputs from the qa chain when the submit button is clicked
  to generate a response to the query"""

  if not btn:
      raise gr.Error(message='Upload a PDF')

  qa_chain_with_memory = process_file(btn) # run the qa chain with files from upload
  bot_response = qa_chain_with_memory({"query": query})
  # simulate streaming
  for char in bot_response['result']:
          history[-1][-1] += char
          time.sleep(0.05)
          yield history,''

# The GRADIO Interface
with gr.Blocks() as demo:
    with gr.Row():
            with gr.Row():
              # Chatbot interface
              chatbot = gr.Chatbot(label="DeciLM-7B-instruct bot",
                                   value=[],
                                   elem_id='chatbot')
            with gr.Row():
              # Uploaded PDFs window
              file_output = gr.File(label="Your PDFs")

              with gr.Column():
                # PDF upload button
                btn = gr.UploadButton("📁 Upload a PDF(s)",
                                      file_types=[".pdf"],
                                      file_count="multiple")

    with gr.Column():
        with gr.Column():
          # Ask question input field
          txt = gr.Text(show_label=False, placeholder="Enter question")

        with gr.Column():
          # button to submit question to the bot
          submit_btn = gr.Button('Ask')

    # Event handler for uploading a PDF
    btn.upload(fn=upload_file, inputs=[btn], outputs=[file_output])

    # Event handler for submitting text question and generating response
    submit_btn.click(
        fn= add_text,
        inputs=[chatbot, txt],
        outputs=[chatbot],
        queue=False
        ).success(
          fn=generate_bot_response,
          inputs=[chatbot, txt, btn],
          outputs=[chatbot, txt]
        ).success(
          fn=upload_file,
          inputs=[btn],
          outputs=[file_output]
        )

if __name__ == "__main__":
    demo.launch() # launch app